# Setup

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform google-cloud-aiplatform[evaluation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.1/738.1 kB 51.8 MB/s eta 0:00:00


In [1]:
import datetime
import nest_asyncio
import pandas as pd
from IPython.display import display, Markdown, HTML

import vertexai
from vertexai.generative_models import GenerativeModel

pd.set_option('display.max_colwidth', None)

In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'NV_LIBCUBLAS_VERSION': '12.5.3.2-1',
        'NVIDIA_VISIBLE_DEVICES': 'all',
        'NV_NVML_DEV_VERSION': '12.5.82-1',
        'NV_CUDNN_PACKAGE_NAME': 'libcudnn9-cuda-12',
        'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.22.3-1+cuda12.5',
        'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.22.3-1',
        'VERTEX_PRODUCT': 'COLAB_ENTERPRISE',
        'VM_GCE_METADATA_HOST': '169.254.169.254',
        'HOSTNAME': '702eae3d2ebe',
        'LANGUAGE': 'en_US',
        'COLAB_TPU_1VM': '',
        'GOOGLE_CLOUD_PROJECT': 'qwiklabs-gcp-00-99f245467a0e',
        'GCE_METADATA_TIMEOUT': '3',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand

In [3]:
PROJECT_ID = "qwiklabs-gcp-04-a60b4d3a4b6f"
LOCATION = "us-west1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
vertexai

<module 'vertexai' from '/usr/local/lib/python3.11/dist-packages/vertexai/__init__.py'>

# Task 2. Explore example data and use it to generate content

In [5]:
# Run this line in a new code cell (by pasting the code and hitting Shift + Return) to download some example data.
!gcloud storage cp gs://partner-genai-bucket/genai065/apartment_table.csv .

Copying gs://partner-genai-bucket/genai065/apartment_table.csv to file://./apartment_table.csv


In [6]:
# Load it into a Pandas DataFrame, and view the first few rows:
apartment_df = pd.read_csv("apartment_table.csv")
apartment_df.head()

,Address,Unit,Sqft,Bedrooms,Elevator,Washer & Dryer in Unit,Pets Allowed,Notable features
0,"123 West 14th Street, New York, NY 10014",2E,550,2,yes,no,yes,"doorman, pool in the building, shared roof deck with grills"
1,"456 East 57th Street, New York, NY 10022",1A,789,1,no,yes,no,"bike room, package service"
2,"789 Broadway, New York, NY 10003",C,999,3,yes,yes,yes,"excellent laundry room, great city views"
3,"1011 5th Avenue, New York, NY 10028",30,1024,2,no,no,yes,"great view of Central Park, high ceilings"
4,"2222 Park Avenue, New York, NY 10017",4F,1234,1,no,yes,no,right next to soccer fields at the park


You should see a table like this, presenting data about apartments in New York City, including how many bedrooms they have, how large they are (measured in square feet, identified in this dataset as “Sqft”), whether the building has an elevator, and other attributes of an apartment.

![table](https://cdn.qwiklabs.com/s3FNOyCHL3J1RtAtEl4Pm4dVkNLBkIejB9ahmBmfrHU%3D)

Notice in particular the “Notable features” column, in which unstructured notes from a real estate agent have been captured to describe other selling points of each apartment. You will have a model generate apartment listing descriptions based on these structured and unstructured fields, and then you will measure how well the model sticks to the information in these fields when generating listing descriptions.



In [8]:
# Restructure the data into a list of records with keys identified for each record, and view the first record.
apartment_records = apartment_df.to_dict(orient='records')
apartment_records[0]

{'Address': '123 West 14th Street, New York, NY 10014',
 'Unit': '2E',
 'Sqft': 550,
 'Bedrooms': 2,
 'Elevator': 'yes',
 'Washer & Dryer in Unit': 'no',
 'Pets Allowed': 'yes',
 'Notable features': 'doorman, pool in the building, shared roof deck with grills'}

In [10]:
# Now it’s time to generate some text.
# Instantiate a generative model, define a prompt with some instructions for it,
# and generate content based on the example data:
model = GenerativeModel(
  "gemini-2.5-pro",
  generation_config={
      "temperature": 0,
      "top_p": 0.4,
  },
)

prompt = "Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: "

# View the response using Markdown to format it nicely for notebook viewing
Markdown(model.generate_content(prompt + str(apartment_records[0])).text)

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


PermissionDenied: 403 Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "qwiklabs-gcp-04-a60b4d3a4b6f"
}
metadata {
  key: "consumer"
  value: "projects/qwiklabs-gcp-04-a60b4d3a4b6f"
}
, locale: "en-US"
message: "Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]

### Hypothesis
The error PermissionDenied: 403 Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f. with the reason CONSUMER_INVALID indicates that your Google Cloud project qwiklabs-gcp-04-a60b4d3a4b6f does not have the necessary API enabled to use Vertex AI's generative models.

Here's a step-by-step explanation of what might have caused the error:
- Vertex AI Initialization: You successfully initialized Vertex AI with vertexai.init(project=PROJECT_ID, location=LOCATION). This step configures the client to interact with Vertex AI services in your specified project and location.
- Model Instantiation: You then instantiated a GenerativeModel using "gemini-2.5-pro". This step prepares the model object but doesn't necessarily make an API call to the generative model service itself.
- Content Generation Request: The error occurs when you call model.generate_content(prompt + str(apartment_records[0])).text. This is the point where your code attempts to send a request to the Vertex AI service to generate content using the Gemini model.
- Permission Denied: The traceback clearly shows PermissionDenied: 403 Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f. The debug_error_string further specifies grpc_message:"Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f." and the detailed error message includes reason: "CONSUMER_INVALID" domain: "googleapis.com" metadata { key: "service" value: "aiplatform.googleapis.com" }.
- Root Cause: The CONSUMER_INVALID reason, combined with the service aiplatform.googleapis.com, strongly suggests that the Vertex AI API (which is aiplatform.googleapis.com) is not enabled for your Google Cloud project qwiklabs-gcp-04-a60b4d3a4b6f. Even if you have the correct permissions, if the API itself is not enabled for the project, any attempt to use its services will result in a permission denied error because the project is not a valid "consumer" of that service.


### Suggested Changes
To fix this error, you need to enable the Vertex AI API for your Google Cloud project. This can be done using the gcloud command-line tool.
```
# Enable the Vertex AI API for your project.
# This command needs to be run once per project.
# After running, you might need to wait a minute or two for the changes to propagate.
!gcloud services enable aiplatform.googleapis.com

# After enabling the API, you can re-run the cell that caused the error.
# The rest of your code should work as intended.
# View the response using Markdown to format it nicely for notebook viewing
# Markdown(model.generate_content(prompt + str(apartment_records[0])).text)
```

In [11]:
!gcloud services enable aiplatform.googleapis.com

In [12]:
# Now it’s time to generate some text.
# Instantiate a generative model, define a prompt with some instructions for it,
# and generate content based on the example data:
model = GenerativeModel(
  "gemini-2.5-pro",
  generation_config={
      "temperature": 0,
      "top_p": 0.4,
  },
)

prompt = "Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: "

# View the response using Markdown to format it nicely for notebook viewing
Markdown(model.generate_content(prompt + str(apartment_records[0])).text)

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


PermissionDenied: 403 This API method requires billing to be enabled. Please enable billing on project #qwiklabs-gcp-04-a60b4d3a4b6f by visiting https://console.developers.google.com/billing/enable?project=qwiklabs-gcp-04-a60b4d3a4b6f then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "BILLING_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "qwiklabs-gcp-04-a60b4d3a4b6f"
}
metadata {
  key: "consumer"
  value: "projects/qwiklabs-gcp-04-a60b4d3a4b6f"
}
, locale: "en-US"
message: "This API method requires billing to be enabled. Please enable billing on project #qwiklabs-gcp-04-a60b4d3a4b6f by visiting https://console.developers.google.com/billing/enable?project=qwiklabs-gcp-04-a60b4d3a4b6f then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to o

The model will have generated some text like what you see below. You can compare the content to the record dictionary you saw above to see that features like the number of bedrooms and the square footage are included. Multiple details from the unstructured notes (the doorman, the pool, the grills on the roof deck) are also included.

```
Live the high life in this stunning 2-bedroom apartment at 123 West 14th Street! This spacious 550 sq ft unit boasts an abundance of natural light and modern finishes. Enjoy the convenience of an elevator building and the luxury of a doorman. Take a dip in the building's sparkling pool or soak up the sun on the shared roof deck with grills. This pet-friendly apartment is your oasis in the heart of the city. Don't miss out on this incredible opportunity!
```

You might need to take caution, however, as the description may also include some information that we didn’t provide. In the output above, the model suggests that there is an "abundance of natural light" and "modern finishes", but you didn’t tell the model those things about this apartment, so that may not be true.

You’ll set up an evaluation to attempt to quantify how grounded, or using only provided information, are the model's responses.



# Task 3. Configure and trigger a model-based evaluation

View the prompt templates for model-based evaluation in the documentation. Select the Pointwise text use case template for "Groundedness" to read the prompt.

https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates

##Pointwise groundedness

```
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing groundedness, which measures the ability to provide or reference information included only in the user prompt.

## Criteria
Groundedness: The response contains information included only in the user prompt. The response does not reference any outside information.

## Rating Rubric
1: (Fully grounded). All aspects of the response are attributable to the context.
0: (Not fully grounded). The entire response or a portion of the response is not attributable to the context provided by the user prompt.

## Evaluation Steps
STEP 1: Assess the response in aspects of Groundedness. Identify any information in the response not present in the prompt and provide assessment according to the criterion.
STEP 2: Score based on the rating rubric. Give a brief rationale to explain your evaluation considering Groundedness.

# User Inputs and AI-generated Response
## User Inputs
### Prompt
{prompt}

## AI-generated Response
{response}
```

Notice that the model will score a response with a 1 if it appears to be fully grounded, or a 0 otherwise.

Also notice that the prompt contains a couple of placeholders in curly braces: `{prompt}` and `{response}`. You must provide at least the prompt data in your evaluation dataset, but you can choose whether to provide the response data yourself or let the Evaluation Task run the prompts through a model you provide to generate the responses before evaluating them.

Additionally, pairwise metrics (which compare two models) require the response of the other model you are comparing against, which must be provided in a field named `baseline_model_response`. For chat evaluations, the chat history will also need to be provided as history.



## Pairwise groundedness

```
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by two AI models. We will provide you with the user input and a pair of AI-generated responses (Response A and Response B).
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will first judge responses individually, following the Rating Rubric and Evaluation Steps.
Then you will give step-by-step explanations for your judgment, compare results to declare the winner based on the Rating Rubric and Evaluation Steps.

# Evaluation
## Metric Definition
You will be assessing groundedness, which measures the ability to provide or reference information included only in the user prompt.

## Criteria
Groundedness: The response contains information included only in the user prompt. The response does not reference any outside information.

## Rating Rubric
"A": Response A is more grounded than Response B.
"SAME": Both response A and B are equally grounded, or ungrounded.
"B": Response B is more grounded than Response A.

## Evaluation Steps
STEP 1: Analyze Response A based on the groundedness criteria: Identify any information in the response not present in the prompt and provide assessment according to the criterion.
STEP 2: Analyze Response B based on the groundedness criteria: Identify any information in the response not present in the prompt and provide assessment according to the criterion.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# User Inputs and AI-generated Responses
## User Inputs
### Prompt
{prompt}

## AI-generated Responses

### Response A
{baseline_model_response}

### Response B
{response}

```

In [13]:
# To create an evaluation dataset, create the prompt for each example, which will consist of the prompt instructions you defined earlier and the context data for each apartment.

# Context is the supplemental information you provide the
# model, usually specific to a given query or example,
# that it needs to fulfill your instructions.
# In this case, the context is each apartment record.
contexts = [str(record) for record in apartment_records]
# The full prompt combines the prompt instructions you
# created earlier with the context for each apartment.
full_prompts = [prompt + str(record) for record in apartment_records]

print(full_prompts[0])


Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: {'Address': '123 West 14th Street, New York, NY 10014', 'Unit': '2E', 'Sqft': 550, 'Bedrooms': 2, 'Elevator': 'yes', 'Washer & Dryer in Unit': 'no', 'Pets Allowed': 'yes', 'Notable features': 'doorman, pool in the building, shared roof deck with grills'}


Instead of generating the responses, yourself, create an evaluation dataset of just the prompts, and the evaluation service will generate responses for you as part of the evaluation task.


Note:Because of Qwiklabs quota limitations, you will limit your evaluation dataset to 5 examples. But the best practice recommendation would be to include around 100 examples covering the types of inputs your model might see.
https://cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-dataset#best-practices


In [20]:
eval_dataset = pd.DataFrame({
    "prompt": full_prompts[0:5],
})

eval_dataset

,prompt
0,"Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: {'Address': '123 West 14th Street, New York, NY 10014', 'Unit': '2E', 'Sqft': 550, 'Bedrooms': 2, 'Elevator': 'yes', 'Washer & Dryer in Unit': 'no', 'Pets Allowed': 'yes', 'Notable features': 'doorman, pool in the building, shared roof deck with grills'}"
1,"Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: {'Address': '456 East 57th Street, New York, NY 10022', 'Unit': '1A', 'Sqft': 789, 'Bedrooms': 1, 'Elevator': 'no', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'no', 'Notable features': 'bike room, package service'}"
2,"Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: {'Address': '789 Broadway, New York, NY 10003', 'Unit': 'C', 'Sqft': 999, 'Bedrooms': 3, 'Elevator': 'yes', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'yes', 'Notable features': 'excellent laundry room, great city views'}"
3,"Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: {'Address': '1011 5th Avenue, New York, NY 10028', 'Unit': '30', 'Sqft': 1024, 'Bedrooms': 2, 'Elevator': 'no', 'Washer & Dryer in Unit': 'no', 'Pets Allowed': 'yes', 'Notable features': 'great view of Central Park, high ceilings'}"
4,"Write a one paragraph apartment listing to promote this apartment. Make it sound amazing: {'Address': '2222 Park Avenue, New York, NY 10017', 'Unit': '4F', 'Sqft': 1234, 'Bedrooms': 1, 'Elevator': 'no', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'no', 'Notable features': 'right next to soccer fields at the park'}"


In [16]:
# Now you'll explore some of the classes available for evaluation. Run the following imports and print the list of available MetricPromptTemplateExamples:
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PairwiseMetricPromptTemplate,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
)

MetricPromptTemplateExamples.list_example_metric_names()


['coherence',
 'fluency',
 'safety',
 'groundedness',
 'instruction_following',
 'verbosity',
 'text_quality',
 'summarization_quality',
 'question_answering_quality',
 'multi_turn_chat_quality',
 'multi_turn_safety',
 'pairwise_coherence',
 'pairwise_fluency',
 'pairwise_safety',
 'pairwise_groundedness',
 'pairwise_instruction_following',
 'pairwise_verbosity',
 'pairwise_text_quality',
 'pairwise_summarization_quality',
 'pairwise_question_answering_quality',
 'pairwise_multi_turn_chat_quality',
 'pairwise_multi_turn_safety']

Pre-built metric templates


pointwise_groundedness
```
pointwise_groundedness = PointwiseMetric(
    metric="Groundedness",
    metric_prompt_template=PointwiseMetricPromptTemplate(
        criteria={
            "Groundedness": "The response contains information included only in the user prompt. The response does not reference any outside information.",
        },
        rating_rubric={
            "1": "(Fully grounded). All aspects of the response are attributable to the context.",
            "0": "(Not fully grounded). The entire response or a portion of the response is not attributable to the context provided by the user prompt.",
        },
        input_variables=[
            "prompt",
            "response",
        ],
    ),
)
```

pairwise_groundedness
```
pairwise_groundedness = PairwiseMetric(
    metric="Groundedness",
    metric_prompt_template=PairwiseMetricPromptTemplate(
        criteria={
            "Groundedness": "The response contains information included only in the user prompt. The response does not reference any outside information.",
        },
        rating_rubric={
            "A": "Response A is more grounded than Response B.",
            "SAME": "Both response A and B are equally grounded, or ungrounded.",
            "B": "Response B is more grounded than Response A.",
        },
        input_variables=[
            "prompt",
            "response",
            "baseline_model_response",
        ],
    ),
    baseline_model=GenerativeModel("gemini-2.5-flash"),
)
```

In [17]:
# Instead of using the documentation to review the prompt as you did earlier,
# you can view the criteria and ratings for each metric within your code like so:
print(MetricPromptTemplateExamples.get_prompt_template('groundedness'))



# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated response.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step by step explanations for your rating, and only choose ratings from the Rating Rubric.


# Evaluation
## Metric Definition
You will be assessing groundedness, which measures the ability to provide or reference information included only in the user prompt.

## Criteria
Groundedness: The response contains information included only in the user prompt. The response does not reference any outside information.

## Rating Rubric
1: (Fully grounded). All aspects of the response are attributable to the context.
0: (Not fully grounde

In [19]:
# Instantiate an EvalTask by associating your dataset & selected metric.
# You can also provide an experiment name to track your evaluations in Vertex AI Experiments.
# https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments

eval_task = EvalTask(
  dataset=eval_dataset,
  metrics=[MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS],
  experiment="apartment-listing-generation",
)

eval_task


In [21]:
# Run the evaluate() method on the task by giving it a unique run name.
# By passing
#   the model you would like to evaluate,
#   the EvalTask can generate the responses needed to complete the evaluation dataset.
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
eval_result = eval_task.evaluate(
  model=model,
  experiment_run_name=f"apt-gen-{run_ts}"
  )


# You might want to keep track of your results in a list
# which you will use to plot your results later on
eval_results_to_compare = []
eval_results_to_compare.append(eval_result)


PermissionDenied: 403 Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "qwiklabs-gcp-04-a60b4d3a4b6f"
}
metadata {
  key: "consumer"
  value: "projects/qwiklabs-gcp-04-a60b4d3a4b6f"
}
, locale: "en-US"
message: "Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]

The evaluation service's results consist of three parts: a complete table of results (metrics_table), some summary metrics with averages and standard deviation of the metrics (summary_metrics), and some metadata related to the run. First look at the summary metrics:

```
eval_result.summary_metrics
```


Viewing the outputs, you'll see that the groundedness was likely not perfect. From the Groundedness prompt descriptions we saw above, we know the model scores a 1 for grounded or a 0 for grounded, so a perfect result would have returned a mean of 1.0 and a std (or standard deviation) of 0 suggesting no deviation from that mean.

Next, view the complete results table:
```
eval_result.metrics_table
```

![results](https://cdn.qwiklabs.com/c69eBjWYrMouC8e4%2FwbMeqLiw0jTWyW93omHb%2ByygJo%3D)

You should see a table like this containing your results. Look for examples where the groundedness/score is 0.0, meaning the results were not grounded. Read the response and then the groundedness/explanation to see why the model rated the response as not grounded. You may see something like:
```
Although the response appropriately incorporates all details from the input, it introduces external details not found in the prompt such as: 'modern finishes,' 'ample natural light,' 'sparkling pool,' 'well-appointed retreat,' 'vibrant heart of New York City,' 'serenity,' 'stylish haven,' 'convenient location,' and 'thoughtful amenities.' Therefore, the response is not fully grounded.
```

# Task 4. Improve the prompt to see scores improve
In this task, you’ll make a tweak to address the hallucination concerns and improve your application’s groundedness score.



Improve the prompt to improve your groundedness score. With the new prompt, also create a new evaluation dataset.


In [22]:
# This is a minor update, but should stop the model from
# inventing as many details about each apartment.
updated_prompt = "Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: "

updated_full_prompts = [updated_prompt + str(record) for record in apartment_records]

updated_eval_dataset = pd.DataFrame(
  {
      "prompt": updated_full_prompts[0:5]
  }
)

In [23]:
updated_full_prompts

["Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '123 West 14th Street, New York, NY 10014', 'Unit': '2E', 'Sqft': 550, 'Bedrooms': 2, 'Elevator': 'yes', 'Washer & Dryer in Unit': 'no', 'Pets Allowed': 'yes', 'Notable features': 'doorman, pool in the building, shared roof deck with grills'}",
 "Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '456 East 57th Street, New York, NY 10022', 'Unit': '1A', 'Sqft': 789, 'Bedrooms': 1, 'Elevator': 'no', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'no', 'Notable features': 'bike room, package service'}",
 "Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '789 Broadway, New York, NY 10003'

In [24]:
updated_eval_dataset

,prompt
0,"Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '123 West 14th Street, New York, NY 10014', 'Unit': '2E', 'Sqft': 550, 'Bedrooms': 2, 'Elevator': 'yes', 'Washer & Dryer in Unit': 'no', 'Pets Allowed': 'yes', 'Notable features': 'doorman, pool in the building, shared roof deck with grills'}"
1,"Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '456 East 57th Street, New York, NY 10022', 'Unit': '1A', 'Sqft': 789, 'Bedrooms': 1, 'Elevator': 'no', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'no', 'Notable features': 'bike room, package service'}"
2,"Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '789 Broadway, New York, NY 10003', 'Unit': 'C', 'Sqft': 999, 'Bedrooms': 3, 'Elevator': 'yes', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'yes', 'Notable features': 'excellent laundry room, great city views'}"
3,"Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '1011 5th Avenue, New York, NY 10028', 'Unit': '30', 'Sqft': 1024, 'Bedrooms': 2, 'Elevator': 'no', 'Washer & Dryer in Unit': 'no', 'Pets Allowed': 'yes', 'Notable features': 'great view of Central Park, high ceilings'}"
4,"Write a one paragraph apartment listing highlighting the best known features of this apartment. Use only the details included in the following information: {'Address': '2222 Park Avenue, New York, NY 10017', 'Unit': '4F', 'Sqft': 1234, 'Bedrooms': 1, 'Elevator': 'no', 'Washer & Dryer in Unit': 'yes', 'Pets Allowed': 'no', 'Notable features': 'right next to soccer fields at the park'}"


In [25]:
# Create a new EvalTask and run its evaluate() method to generate evaluations.
# Preview them in a table.
# Your groundedness/mean score should now have increased.

updated_eval_task = EvalTask(
  dataset=updated_eval_dataset,
  metrics=[MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS],
  experiment="apartment-listing-generation",
)

run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
updated_result = updated_eval_task.evaluate(
  model=model,
  experiment_run_name=f"apt-gen-{run_ts}"
)

# Append the new result to your results
eval_results_to_compare.append(updated_result)

# Preview the summary
print(updated_result.summary_metrics)

PermissionDenied: 403 Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "qwiklabs-gcp-04-a60b4d3a4b6f"
}
metadata {
  key: "consumer"
  value: "projects/qwiklabs-gcp-04-a60b4d3a4b6f"
}
, locale: "en-US"
message: "Permission denied on resource project qwiklabs-gcp-04-a60b4d3a4b6f."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]

Use the provided helper function to visualize your evaluation runs against each other. See whether your application improved on the metrics:

```
import plotly.graph_objects as go
def plot_bar_plot(eval_results, metrics=None):
  fig = go.Figure()
  data = []
  for eval_result in eval_results:
      summary_metrics = eval_result.summary_metrics
      if metrics:
          summary_metrics = {
              k: summary_metrics[k]
              for k, v in summary_metrics.items()
              if any(selected_metric in k for selected_metric in metrics)
          }


      data.append(
          go.Bar(
              x=list(summary_metrics.keys()),
              y=list(summary_metrics.values()),
              name=eval_result.metadata["experiment_run"]
          )
      )
  fig = go.Figure(data=data)


  # Change the bar mode
  fig.update_layout(barmode="group")
  fig.show()


plot_bar_plot(eval_results_to_compare, metrics=["groundedness/mean"])
```

You should see that you’ve made a measurable improvement on groundedness!

![bar chart](https://cdn.qwiklabs.com/ON2grjneG96wSE6%2Bea9I2h2dj%2FIKwHzXjCIRRrjVCA0%3D)

Within the Google Cloud Console, navigate to Vertex AI and then Experiments within the left navigation bar. If you click on your experiment name and select a run name, you can select the Metrics and Parameters tabs to see that the Generative AI Evaluation Service has documented your runs.
